Movie Recommendation engine
1. your recommended engine will return maximum 10 movies name if an user search for a particular movie.
Evaluation

2. Recommended engine must return 5 movie names and maximum it can return 10 movie names if an user search for a particular movie. This recommender engine should not give suggestion in between 1 to 4 and 6 to 10 it have to return 5 movie names or 10 movie names.

Analysis

1. This will be a content-based filtering which make recomemndation based on the attributes

2. As a netflix binge watcher, the category are very well separated by the type, so we will want recommendation based on the type, i.e TV shows or Movies

3. Since the current content involve alot of unqiue features such as director, genre, country and casts, we will use a CountVectorizer rather than a TfidfVectorizer which is meant for eliminating any repeated words. 

In [ ]:
import numpy as np
import pandas as pd
import re
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
data = pd.read_csv('/kaggle/input/netflix-shows/netflix_titles.csv')
data.head(15)

In [ ]:
# Understanding the data
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
# Drop missing data in the excel, it is a personal choice not to drop country cause tbh I will not look at the country for recommendation
data = data.dropna(subset=['director','cast', 'rating'])
data.shape

In [ ]:
data = data.drop(['show_id','date_added','release_year','duration','description'], axis = 1)
data.head()

In [ ]:
data.isnull().sum()

In [ ]:
data.groupby('type').count()

In [ ]:
# splitting the data, perfect place to use the Regular expression
data['listed_in'] = [re.sub(r'[^\w\s]', '', t) for t in data['listed_in']]
data['cast'] = [re.sub(r'[^\w]', '', t) for t in data['cast']]

data['listed_in'] = [t.lower() for t in data['listed_in']]
data['cast'] = [t.lower() for t in data['cast']]

# creating bag of words
data["combined"] = data['listed_in'] + '  ' + data['title'] + ' ' + data['cast'] + ' ' + data['country'] 
data.drop(["cast", "listed_in", "country", 'director'],axis=1,inplace=True)
data.head()

In [ ]:
#separate tv shows and movies
Tv = data[data['type'] =='TV Show'].reset_index()
Mov = data[data['type'] =='Movie'].reset_index()
Tv.head()

In [ ]:
# main

vectorizer = CountVectorizer()
recommended_movies = []

# matrixTv = vectorizer.fit_transform(Tv['combined'].values.astype('U'))
# idx = Tv[Tv['title'] == search].index.item()
# similar = cosine_similarity(matrix,matrix)
# score = pd.Series(similar[idx]).sort_values(ascending = False)
# top_10 = list(score.iloc[1:11].index)

# for i in top_10:
#     recommended_movies.append((Tv['title'])[i])
# recommended_movies

def netflix_recommender(search):
    if search in Tv['title'].values:
        matrixTv = vectorizer.fit_transform(Tv['combined'].values.astype('U'))
        #find the index of the item been searched
        idx = Tv[Tv['title'] == search].index.item()
        similar = cosine_similarity(matrixTv,matrixTv)
        score = pd.Series(similar[idx]).sort_values(ascending = False)
        top_10 = list(score.iloc[1:11].index)
        
        for i in top_10:
            recommended_movies.append((Tv['title'])[i])
        return recommended_movies
    
    elif search in Mov['title'].values:
        matrixMov = vectorizer.fit_transform(Mov['combined'].values.astype('U'))
        idx = Mov[Mov['title'] == search].index.item()
        similar = cosine_similarity(matrixMov,matrixmov)
        score = pd.Series(similar[idx]).sort_values(ascending = False)
        top_10 = list(score.iloc[1:11].index)
        
        for i in top_10:
            recommended_movies.append((Mov['title'])[i])
            
        return recommended_movies
    
    else :
        print("Error! No such title found")
        

In [ ]:
netflix_recommender('Almost Happy')

Future work : We can also use the Description column and combine it with the current recommendation egine, perhaps, we can use Tfidf on the description and enumerate to the code to take a combined score from both as the information within the description provide some insight such as adjective and keywords for viewers that attract them to watch the movie as well